In [8]:
"""
Read in the metadata and merge with the novelty data
"""

data_dir = 'lit_data/precocity'
all_dfs = []

import pandas as pd
import os
import re
# Load metadata
df_meta = pd.read_csv('lit_data/lit_metadata/LitMetadataWithS2.tsv', sep='\t')

# Load all paper data files
print("Reading data files...")
for file in os.listdir(data_dir):
    if file.endswith('.tsv'):
        match = re.search(r'(\d{4})', os.path.basename(file))
        year = match.group(1) if match else 'unknown'
        # print(f"Processing {csv} (year: {year})")
        path = os.path.join(data_dir, file)
        df = pd.read_csv(path, sep='\t')
        df['paperId'] = df['docid']
        df['decade'] = year
        merged = df_meta.merge(df, on='paperId', how='left')
        all_dfs.append(merged)


gender_meta = pd.read_csv('lit_data/author_genders_lit.csv')
#read in the gender data we have already


Reading data files...


In [9]:
df_all = pd.concat(all_dfs, ignore_index=True)


In [10]:
"deduplicate the data by selecting one set of parameters ted recommended"

#filter for only male/female gender and also for the single set of novelty results (time chunks etc)

#chunks_used
    #use 0.25 (top 25% most novel chunks)
df_all = df_all.loc[df_all['chunks_used'] == 0.25]

#time_radius
    #use 20
df_all = df_all.loc[df_all['time_radius'] == 20]


#filtered
    #use trainauthquote
df_all = df_all.loc[df_all['filtered'] == 'trainauthquote']



#fraction_compared
    #10 percent most similar vs all articles... use all articles aka 1.0

df_all = df_all.loc[df_all['fraction_compared'] == 1.0]


#num_chunks
    #I don't think this one needed to be filtered?

In [12]:
gender_meta = gender_meta[gender_meta['gender'].isin(['male', 'female'])]


In [16]:
#apply gender dict
gender_dict = pd.Series(gender_meta['gender'].values, index=gender_meta['author']).to_dict()
df_all['gender'] = df_all['authors'].map(gender_dict).fillna('unknown')

In [17]:
df_all.head()

,doi,journal,year,authors,title,language,wordcount,doctype,citation_counts,S2titles,...,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,decade,gender
17,10.2307/461288,PMLA,1967,['Jean-Jacques Demorest'],Pascal et le déséquilibre,['fre'],4999,article | research-article,0,Pascal et le déséquilibre,...,7.0,1.0,trainauthquote,20.0,0.25,-0.013390,8.627299,8.640689,1960,unknown
68,10.2307/3723455,The Modern Language Review,1969,['Patricia Thomson'],Review Article,['eng'],1243,article | book-review,8,The English Petrarchans : a critical bibliogra...,...,2.0,1.0,trainauthquote,20.0,0.25,0.284837,5.996719,5.711883,1960,unknown
174,10.2307/3721313,The Modern Language Review,1963,['Margaret McHaffie'],Review Article,['eng'],600,article | book-review,0,Jeremias Gotthelf : eine Einführung in seine W...,...,1.0,1.0,trainauthquote,20.0,0.25,0.235353,6.185017,5.949665,1960,unknown
339,10.2307/460722,PMLA,1963,['Lillian Herlands Hornstein'],King Robert of Sicily: A New Manuscript,['eng'],3991,article | research-article,0,King Robert of Sicily: A New Manuscript,...,7.0,1.0,trainauthquote,20.0,0.25,0.112459,6.578631,6.466172,1960,unknown
404,10.2307/512220,The Review of English Studies,1967,['William H. Matchett'],Donne's 'Peece of Chronicle',['eng'],920,article | research-article,0,DONNE'S ’PEECE OF CHRONICLE‘,...,1.0,1.0,trainauthquote,20.0,0.25,0.280793,5.364418,5.083625,1960,unknown


In [18]:
def split_list(s):
    return s.split(';')


df_all['authors'] = df_all['authors'].map(split_list)



In [19]:
# df_all['creator_list'].explode()
# df_all['creator'] = df_all['creator'].explode()

df_all = df_all.explode('authors')


In [21]:
df_all['author_clean'] = df_all['authors'].str.replace(r'<prefix>.*?</prefix>', '', regex=True).str.strip()

In [22]:
df_all['first_name'] = df_all['author_clean'].str.split().str[0]


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# df_all['gender'] = df_all['gender'].dropna()

df_all = df_all.dropna(subset=['gender'])
df_all = df_all.dropna(subset=['first_name'])

df_all = df_all.loc[df_all['gender'] != 'unknown']

# df should contain columns: "first_name" and "gender"
X = df_all["first_name"]
y = df_all["gender"]

model = Pipeline([
    ("vect", CountVectorizer(analyzer="char", ngram_range=(2,3))),  # character n-grams
    ("clf", LogisticRegression(max_iter=500))
])

model.fit(X, y)


KeyError: ['first_name']